 aqui se preparan los conjuntos de datos necesarios para hacer las consultas en la API.

In [1]:
import pandas as pd
# import pyarrow as pa
# import pyarrow.parquet as pq



In [2]:

df_games = pd.read_parquet(r'./Data/clean_games_steam.parquet.gzip')
df_items = pd.read_parquet(r'./Data/clean_games_items.parquet.gzip') 
df_reviews = pd.read_parquet(r'./Data/clean_games_reviews.parquet.gzip') 


primer endpoint , PlayTimeGenre; Debe devolver `año` con mas horas jugadas para dicho género.

Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [27]:
PlayTimeGenre = df_games[['genres', 'year', 'developer', 'id']]
df_items_aux = df_items[['playtime_forever']]

playTimeGenre = pd.concat([df_items_aux], axis=1)
# Se renombra la columna 'id' para unirla con el dataframe anterior
PlayTimeGenre = PlayTimeGenre.rename(columns={'id':'item_id'})
PlayTimeGenre = PlayTimeGenre.rename(columns={'':'playtime_forever'})
# se eliminan los duplicados
PlayTimeGenre = PlayTimeGenre.drop_duplicates()

# Reseteamos el indice
# PlayTimeGenre = PlayTimeGenre.reset_index(drop=True, inplace=True)
PlayTimeGenre.head()


,genres,year,developer,item_id
88310,Action,2018,Kotoshiro,761140.0
88310,Casual,2018,Kotoshiro,761140.0
88310,Indie,2018,Kotoshiro,761140.0
88310,Simulation,2018,Kotoshiro,761140.0
88310,Strategy,2018,Kotoshiro,761140.0


In [8]:
PlayTimeGenre.to_parquet(r'./Data/endpoint1.parquet.gzip', compression='gzip') # Exportamos


Segundo endpoint.
UserForGenre( *`genero` : str* )**:
    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf,
			     "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [28]:
UserForGenre = df_games[['genres', 'year', 'developer', 'id']]


In [29]:
UserForGenre.to_parquet(r'./Data/endpoint2.parquet.gzip', compression='gzip') # Exportamos


Endpoint 3 
UsersRecommend( *`año` : int* )**:
   Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
   Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [16]:
UsersRecommend = df_reviews[['reviews_recommend','reviews_helpful','reviews_review']]


In [17]:
UsersRecommend.to_parquet(r'./Data/endpoint3.parquet.gzip', compression='gzip') # Exportamos



Endpoint 4.

UsersWorstDeveloper( *`año` : int* )**:
   Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
   Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]
  

In [18]:
df_worst_developer_year = df_reviews[['reviews_recommend','reviews_helpful','reviews_review']]


In [19]:
df_worst_developer_year.to_parquet(r'./Data/endpoint4.parquet.gzip', compression='gzip') # Exportamos


Endpoint 5 

sentiment_analysis( *`empresa desarrolladora` : str* )**:
    Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total 
    de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor. 

Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}

 El objetivo es asignar un valor numérico a un texto, en este caso a los comentarios que los usuarios dejaron para un juego determinado, para representar si el sentimiento expresado en el texto es negativo, neutral o positivo."

In [20]:
from textblob import TextBlob
import re


In [21]:
def functionsentiment_analisis(review):
    '''
    Realiza un análisis de sentimiento en un texto dado y devuelve un valor numérico que representa el sentimiento.

    Esta función utiliza la librería TextBlob para analizar el sentimiento en un texto dado y
    asigna un valor numérico de acuerdo a la polaridad del sentimiento.

    Parameters:
        review (str): El texto que se va a analizar para determinar su sentimiento.

    Returns:
        int: Un valor numérico que representa el sentimiento del texto:
             - 0 para sentimiento negativo.
             - 1 para sentimiento neutral o no clasificable.
             - 2 para sentimiento positivo.
    '''
    if review is None:
        return 1
    analysis = TextBlob(review)
    polarity = analysis.sentiment.polarity
    if polarity < -0.2:
        return 0  
    elif polarity > 0.2: 
        return 2 
    else:
        return 1 


In [22]:

df_reviews['sentiment_analysis'] = df_reviews['reviews_review'].apply(functionsentiment_analisis)
df_reviews.head()




,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,1
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,1
2,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,2
3,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2
4,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,Git gud,1


In [30]:
df_developer_reviews_analysis = df_reviews.drop(columns=['reviews_review'])
df_developer_reviews_analysis


,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True
2,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True
3,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True
4,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True
...,...,...,...,...,...
231919,SKELETRONPRIMEISOP,http://steamcommunity.com/id/SKELETRONPRIMEISOP,440,No ratings yet,True
231921,76561198141079508,http://steamcommunity.com/profiles/76561198141...,304930,No ratings yet,True
232047,ShadowYT100,http://steamcommunity.com/id/ShadowYT100,265630,No ratings yet,True
232127,bestcustomurlevermade,http://steamcommunity.com/id/bestcustomurlever...,304050,No ratings yet,True


In [24]:
df_developer_reviews_analysis.to_parquet(r'./Data/endpoint5.parquet.gzip', compression='gzip') # Exportamos


enpoint 6 
Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado

In [25]:
df_recomendacion_juego = df_games[['app_name', 'id','genres']]
df_recomendacion_juego


,app_name,id,genres
88310,lost summoner kitty,761140.0,Action
88310,lost summoner kitty,761140.0,Casual
88310,lost summoner kitty,761140.0,Indie
88310,lost summoner kitty,761140.0,Simulation
88310,lost summoner kitty,761140.0,Strategy
...,...,...,...
120442,russian roads,610660.0,Indie
120442,russian roads,610660.0,Racing
120442,russian roads,610660.0,Simulation
120443,exit 2 - directions,658870.0,Casual


In [26]:
df_recomendacion_juego.to_parquet(r'./Data/endpoint6.parquet.gzip', compression='gzip') # Exportamos
